In [1]:
import os
import pickle
import argparse
import uproot
import awkward as ak
import hist
import pandas as pd

In [7]:
class Args:
    input_file = '../CLUE_clusters_single.root'
    output_file = '/home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/plots/cache/hists.pkl'
args = Args()

In [8]:
for array_i in uproot.iterate(args.input_file + ":clusters", step_size="100MB", library="ak"):
    array = array_i
    break

In [10]:
clusters_2d = ak.to_dataframe(array[
    ["beamEnergy", "NRechits", "clus2D_x", "clus2D_y", "clus2D_z", "clus2D_energy", "clus2D_layer",
        "clus2D_rho", "clus2D_delta", "clus2D_idxs", "clus2D_isSeed"]
    ], 
    levelname=lambda i : {0 : "event", 1:"clus2D_id", 2:"hit_id"}[i])

In [11]:
clusters_3d = ak.to_dataframe(array[
    ["beamEnergy", "NRechits", "clus3D_x", "clus3D_y", "clus3D_z", "clus3D_energy", "clus3D_size", "clus3D_idxs"]
    ], 
    levelname=lambda i : {0 : "event", 1:"clus3D_id", 2:"hit_id"}[i])

In [18]:
 #Merge clusters3D and clusters2D
clusters_3d_2d = pd.merge(
    #Left : clusters3d
    # Reset multiindex so its columns can be kept in joined dataframe (otherwise clus3D_id column disappears)
    clusters_3d.reset_index(level=("clus3D_id", "hit_id"), names=["event", "clus3D_id", "clus3D_hit_id"]),

    #Right : clusters_2d
    # We don't care about rechits so we slice the df by taking the row of first rechit of 2D cluster
    #                  event     cluster2d_id  index of hit inside cluster2D
    # Also reset index for same reason
    clusters_2d.loc[(slice(None), slice(None),   0                       )].reset_index(level="event"),
    how='inner',

    # Map event on both sides
    # Map clus3D_idxs to clus2D_id
    left_on=('event', 'clus3D_idxs'),
    right_on=('event', 'clus2D_id'),

    suffixes=('', '_clus2D') # This is to avoid beamEnergy column (which exists on both sides) to get renamed. We just keep the one from the left
)
clusters_3d_2d

,event,clus3D_id,clus3D_hit_id,beamEnergy,NRechits,clus3D_x,clus3D_y,clus3D_z,clus3D_energy,clus3D_size,...,NRechits_clus2D,clus2D_x,clus2D_y,clus2D_z,clus2D_energy,clus2D_layer,clus2D_rho,clus2D_delta,clus2D_idxs,clus2D_isSeed
0,0,0,0,20.0,317,-2.923339,3.561614,20.577499,0.070715,1,...,317,-2.923340,3.561614,20.577499,0.070715,6,0.070715,3.402823e+38,58,1
1,0,1,0,20.0,317,-1.650613,0.120992,22.685703,15.710234,16,...,317,-1.867480,0.558911,13.877500,0.448077,1,1.254791,3.874668e-01,0,0
2,0,1,1,20.0,317,-1.650613,0.120992,22.685703,15.710234,16,...,317,-1.652880,-0.064618,14.767500,0.684047,2,1.978301,1.593665e-01,5,0
3,0,1,2,20.0,317,-1.650613,0.120992,22.685703,15.710234,16,...,317,-1.869435,0.171449,16.782499,0.929381,3,2.949999,1.610287e-01,13,0
4,0,1,3,20.0,317,-1.650613,0.120992,22.685703,15.710234,16,...,317,-1.758617,0.054618,17.672501,1.211050,4,3.599792,2.032689e-01,23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364344,11603,13,0,20.0,378,-0.398342,2.480419,46.412498,0.079123,1,...,378,-0.398342,2.480419,46.412498,0.079123,23,0.079123,1.586731e+00,275,1
364345,11603,14,0,20.0,378,-2.018789,0.147382,46.412498,0.096182,1,...,378,-2.018789,0.147382,46.412498,0.096182,23,0.096182,3.402823e+38,277,1
364346,11603,15,0,20.0,378,-2.923340,3.937988,46.412498,0.062449,1,...,378,-2.923340,3.937988,46.412498,0.062449,23,0.062449,1.927641e+00,279,1
364347,11603,16,0,20.0,378,-0.000009,1.125244,52.881496,0.111305,1,...,378,-0.000009,1.125244,52.881500,0.111305,27,0.111305,3.402823e+38,285,1


In [25]:
total_clustered_energy_per_layer_clue3D = clusters_3d_2d.groupby(["event", "clus3D_id", "clus2D_layer"]).agg({"clus2D_energy":"sum", "beamEnergy":"first"})
total_clustered_energy_per_layer_clue3D

clus2D_energy  beamEnergy
event clus3D_id clus2D_layer                           
0     0         6                  0.070715        20.0
      1         1                  0.448077        20.0
                2                  0.684047        20.0
                3                  0.929381        20.0
                4                  1.211050        20.0
...                                     ...         ...
11603 13        23                 0.079123        20.0
      14        23                 0.096182        20.0
      15        23                 0.062449        20.0
      16        27                 0.111305        20.0
      17        27                 0.072079        20.0

[362235 rows x 2 columns]

In [27]:
total_clustered_energy_per_layer_clue3D.reset_index("clus2D_layer")

clus2D_layer  clus2D_energy  beamEnergy
event clus3D_id                                         
0     0                     6       0.070715        20.0
      1                     1       0.448077        20.0
      1                     2       0.684047        20.0
      1                     3       0.929381        20.0
      1                     4       1.211050        20.0
...                       ...            ...         ...
11603 13                   23       0.079123        20.0
      14                   23       0.096182        20.0
      15                   23       0.062449        20.0
      16                   27       0.111305        20.0
      17                   27       0.072079        20.0

[362235 rows x 3 columns]

In [30]:
clusters3d_slice = clusters_3d.loc[(slice(None), slice(None), 0)]
# Build an index that selects for each event the 3D cluster with highest energy
index_largest_3D_cluster = clusters3d_slice.groupby(["event"])['clus3D_energy'].transform(max) == clusters3d_slice["clus3D_energy"]
# Apply this index to the df with total clustered energy per layer and per 3D cluster, selecting only the correct 3D cluster
total_clustered_energy_per_layer_highest_clue3D = total_clustered_energy_per_layer_clue3D.reset_index(level=2)[index_largest_3D_cluster]

/tmp/ipykernel_301434/1425608867.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total_clustered_energy_per_layer_highest_clue3D = total_clustered_energy_per_layer_clue3D.reset_index(level=2)[index_largest_3D_cluster]
